In [115]:
import numpy as np
import skimage.io as io
import matplotlib.pyplot as plt
%matplotlib inline

In [144]:
im = io.imread('img.png')

In [116]:
def alignn(im1, im2, limit):
    x_c=0
    y_c=0
    c_val = -np.inf
    [H,W] = np.shape(im2)
    x_coord = np.arange(0,W,1)
    y_coord = np.arange(0,H,1)
    for y in range(-limit,limit+1):
        for x in range(-limit,limit+1):
            x_coord1 = x_coord[(x_coord+x>=0) & (x_coord+x<W)]
            y_coord1 = y_coord[(y_coord+y>=0) & (y_coord+y<H)]
            im22 = im2[np.ix_(y_coord1,x_coord1)]
#             im22 = np.roll(np.roll(im2, -y, axis = 0), -x, axis = 1)
            if x<0:
                x1 = 0
                x2 = W+x
                if y<0:
#                     im11 = im1[0:H+y, 0:W+x]
                    y1=0
                    y2=H+y
                else:
#                     im11 = im1[y:, 0:W+x]
                    y1 = y
                    y2 = im1.shape[0]
            elif x>=0: 
                x1 = x
                x2 = im1.shape[1]
                if y<0:
#                     im11 = im1[0:H+y, x:]
                    y1=0
                    y2=H+y
                else:
#                     im11 = im1[y:, x:]
                    y1 = y
                    y2 = im1.shape[0]
            im11 = im1[y1:y2, x1:x2]        
# #             mse = np.mean((im11 - im22) ** 2)
# #             if m_val > mse:
# #                 m_val = mse
# #                 x_m = x
# #                 y_m = y 
           
            cor = corr(im11,im22)
            if c_val < cor:
                c_val = cor
                x_c = x
                y_c = y
#     print(y_m, x_m)
#     print('y_c_x_c',y_c, x_c)
    return [y_c,x_c]

In [117]:
def im_resize(im, c):
    imag = im
    while(c > 0):
        imag = imag[::2,::2]
        c-=1
    return imag

In [118]:
def my_range(start, end, step):
    while start >= end:
        yield start
        start += step

In [119]:
def corr(im11, im22):
    cor = np.mean((im11 - im11.mean()) * (im22 - im22.mean()))
#     stds = im11.std() * im22.std()
#     cor /= stds
    return np.max(cor)

In [120]:
def mse(im11, im22):
    return np.mean((im11 - im22) ** 2)

In [121]:
def run_single_test(data_dir, output_dir):
#     from align import align
    from skimage.io import imread, imsave
    parts = open(join(data_dir, 'g_coord.csv')).read().rstrip('\n').split(',')
    g_coord = (int(parts[0]), int(parts[1]))
    img = imread(join(data_dir, 'img.png'), plugin='matplotlib')
    start = time()
    aligned_img, (b_row, b_col), (r_row, r_col) = align(img, g_coord)
    end = time()
    running_time = end - start
    with open(join(output_dir, 'output.csv'), 'w') as fhandle:
        print('%d,%d,%d,%d' % (b_row, b_col, r_row, r_col), file=fhandle)

    imsave(join(output_dir, 'aligned_img.png'), aligned_img)
    return running_time


def check_test(output_dir, gt_dir):

    with open(join(output_dir, 'output.csv')) as fhandle:
        parts = fhandle.read().rstrip('\n').split(',')
        b_row, b_col, r_row, r_col = map(int, parts)

    with open(join(gt_dir, 'gt.csv')) as fhandle:
        parts = fhandle.read().rstrip('\n').split(',')
        coords = map(int, parts[1:])
        gt_b_row, gt_b_col, _, _, gt_r_row, gt_r_col, diff_max = coords

    diff = abs(b_row - gt_b_row) + abs(b_col - gt_b_col) + \
        abs(r_row - gt_r_row) + abs(r_col - gt_r_col)

    if diff > diff_max:
        return 'Wrong answer'
    return 'Ok'

In [122]:
def align(im, g_coord):
    [Height,Width]=np.shape(im);
    Height = int(np.rint(Height/3));
    delta_x = int(np.rint(Width*0.05))
    delta_y = int(np.rint(Height*0.05))
    im1 = im[delta_y:(Height-delta_y),delta_x:(Width-delta_x)]
#     print(im1.shape)
    im2 = im[Height+delta_y:(2*Height-delta_y),delta_x:(Width-delta_x)]
#     print(im2.shape)
    im3 = im[2*Height +delta_y:(3*Height-delta_y),delta_x:(Width-delta_x)]
#     print(im3.shape)
    image = np.array([im1,im2,im3[0:im1.shape[0],:]])
    [imm1,imm2,imm3] = image
#     print(image.shape)
    
    [H,W] = image[0,:,:].shape
    k=1
    height = H;
    width = W;
    while (height>500 and width>500):
        height/=2
        width/=2
        k+=1
#     print(k)    
#     print('Processing...')    
    limit=16
    t_x=0
    t_y=0
    y_c=0
    x_c=0
    t_xx=0
    t_yy=0
    y_cc=0
    x_cc=0
    if k == 1:
        [t_y,t_x]=alignn(image[0,:,:], image[1,:,:], 15)
        [t_yy,t_xx]=alignn(image[2,:,:], image[1,:,:], 15)
    else:
        for i in my_range(k-1,2,-1):
            im1 = im_resize(imm1,i)
            im3 = im_resize(imm3,i)
            im2 = im_resize(imm2,i)
            
            [y_c,x_c] = alignn(im1, im2, limit)
            imm1 = np.roll(np.roll(imm1, (-y_c*(2**i)), axis = 0), (-x_c*(2**i)), axis = 1)
            t_x = t_x + x_c*(2**i)
            t_y = t_y + y_c*(2**i)
            
            [y_cc,x_cc] = alignn(im3, im2, limit)
            imm3 = np.roll(np.roll(imm3, (-y_cc*(2**i)), axis = 0), (-x_cc*(2**i)), axis = 1)
            t_xx = t_xx + x_cc*(2**i)
            t_yy = t_yy + y_cc*(2**i)
            limit = int(limit/2)

    [b_row, b_col] = [g_coord[0]-Height+t_y, g_coord[1]+t_x] 
    [r_row, r_col] = [g_coord[0]+Height+t_yy, g_coord[1]+t_xx]

    aligned_img = np.dstack((imm3, image[1,:,:], imm1))

    return aligned_img, (b_row, b_col), (r_row, r_col)

In [124]:
import os
from time import time
for it in range(0,10):
# it=10
    input_dir = ''.join(['/Users/anoshin_alexey/Desktop/Computer Vision/public_data/0',str(it),'_input'])
    dirr = ''.join(['/Users/anoshin_alexey/Desktop/Computer Vision/public_data/0', str(it),'_output'])
    gt_dir = ''.join(['/Users/anoshin_alexey/Desktop/Computer Vision/public_data/0', str(it),'_gt'])
#     os.mkdir(dirr) 
    start = time()
    run_single_test(input_dir, dirr)
    end = time()
    running_time = end - start
    print(running_time)
#     run_single_test(input_dir, dirr)
    print(check_test(dirr, gt_dir))

/anaconda/lib/python3.6/site-packages/skimage/util/dtype.py:122: UserWarning: Possible precision loss when converting from float32 to uint8
  .format(dtypeobj_in, dtypeobj_out))


3.3445918560028076
Ok
2.415203809738159
Ok
2.578166961669922
Ok
3.512716054916382
Ok
2.998234987258911
Ok
3.1765267848968506
Ok
3.241276741027832
Ok
2.6115620136260986
Ok
3.122073173522949
Ok
2.978602886199951
Ok


In [113]:
from time import time
import os
from os.path import join
it=12
input_dir = ''.join(['/Users/anoshin_alexey/Desktop/Computer Vision/public_data/',str(it),'_input'])
dirr = ''.join(['/Users/anoshin_alexey/Desktop/Computer Vision/public_data/', str(it),'_output'])
gt_dir = ''.join(['/Users/anoshin_alexey/Desktop/Computer Vision/public_data/', str(it),'_gt'])
# os.mkdir(dirr) 
# start = time()
# run_single_test(input_dir, dirr)
# end = time()
# running_time = end - start
print(run_single_test(input_dir, dirr))
print(check_test(dirr, gt_dir))

/anaconda/lib/python3.6/site-packages/skimage/util/dtype.py:122: UserWarning: Possible precision loss when converting from float32 to uint8
  .format(dtypeobj_in, dtypeobj_out))


12.676955938339233
Ok


In [104]:
import numpy as np
import skimage.io as io

def alignn(im1, im2, limit):
    x_m=0
    y_m=0
    x_c=0
    y_c=0
    m_val = np.inf
    c_val = -np.inf
    [H,W] = np.shape(im2)
    for y in range(-limit,limit+1):
        for x in range(-limit,limit+1):
            #im11 =  np.zeros(np.shape(im1))-1
            x_coord = np.arange(0,W,1)
            y_coord = np.arange(0,H,1)
            x_coord = x_coord[(x_coord+x>=0) & (x_coord+x<W)]
            y_coord = y_coord[(y_coord+y>=0) & (y_coord+y<H)]
            im22 = im2[np.ix_(y_coord,x_coord)]
            if x<0:
                if y<0:
                    im11 = im1[0:H+y, 0:W+x]
                else:
                    im11 = im1[y:, 0:W+x]
            elif x>=0: 
                if y<0:
                    im11 = im1[0:H+y, x:]
                else:
                    im11 = im1[y:, x:]
#             mse = np.mean((im11 - im22) ** 2)
#             if m_val > mse:
#                 m_val = mse
#                 x_m = x
#                 y_m = y 
            cor = corr(im11,im22)
            if c_val < cor:
                c_val = cor
                x_c = x
                y_c = y
#     print(y_m, x_m)
#     print('y_c_x_c',y_c, x_c)
    return [y_c,x_c]
    
def im_resize(im, c):
    imag = im
    while(c > 0):
        imag = imag[::2,::2]
        c-=1
    return imag
    
def my_range(start, end, step):
    while start >= end:
        yield start
        start += step
        
def corr(im11, im22):
    cor = np.mean((im11 - im11.mean()) * (im22 - im22.mean()))
    stds = im11.std() * im22.std()
    cor /= stds
    return np.max(cor)
    
def mse(im11, im22):
    return np.mean((im11 - im22) ** 2)

def align(im, g_coord):
    [Height,Width]=np.shape(im);
    Height = int(np.rint(Height/3));
    delta_x = int(np.rint(Width*0.05))
    delta_y = int(np.rint(Height*0.05))
    im1 = im[delta_y:(Height-delta_y),delta_x:(Width-delta_x)]
#     print(im1.shape)
    im2 = im[Height+delta_y:(2*Height-delta_y),delta_x:(Width-delta_x)]
#     print(im2.shape)
    im3 = im[2*Height +delta_y:(3*Height-delta_y),delta_x:(Width-delta_x)]
#     print(im3.shape)
    image = np.array([im1,im2,im3[0:im1.shape[0],:]])
    [imm1,imm2,imm3] = image
#     print(image.shape)
    
    [H,W] = image[0,:,:].shape
    k=1
    height = H;
    width = W;
    while (height>500 and width>500):
        height/=2
        width/=2
        k+=1
#     print(k)    
#     print('Processing...')    
    limit=16
    t_x=0
    t_y=0
    y_c=0
    x_c=0
    if k == 1:
        [t_y,t_x]=alignn(image[0,:,:], image[1,:,:], 15)
    else:
        for i in my_range(k-1,2,-1):
            im1 = im_resize(imm1,i)
#             im1 = np.roll(np.roll(im1, (y_c*2), axis = 0), (x_c*2), axis = 1)
            im2 = im_resize(imm2,i)
            [y_c,x_c] = alignn(im1, im2, limit)
            imm1 = np.roll(np.roll(imm1, (-y_c*(2**i)), axis = 0), (-x_c*(2**i)), axis = 1)
            t_x = t_x + x_c*(2**i)
            t_y = t_y + y_c*(2**i)
            limit = int(limit/2)
#             print('t_y_t_x',t_y,t_x)
    im_1 = np.roll(np.roll(image[0,:,:], -t_y, axis = 0), -t_x, axis = 1) 
    [b_row, b_col] = [g_coord[0]-Height+t_y, g_coord[1]+t_x] 
    limit=16
    t_x=0
    t_y=0
    y_c=0
    x_c=0
    if k == 1:
        [t_y,t_x]=alignn(image[2,:,:], image[1,:,:], 15)
    else:
        for i in my_range(k-1,2,-1):
            im1 = im_resize(imm3,i)
#             im1 = np.roll(np.roll(im1, (y_c*2), axis = 0), (x_c*2), axis = 1)
            im2 = im_resize(imm2,i)
            [y_c,x_c] = alignn(im1, im2, limit)
            imm3 = np.roll(np.roll(imm3, (-y_c*(2**i)), axis = 0), (-x_c*(2**i)), axis = 1)
            t_x = t_x + x_c*(2**i)
            t_y = t_y + y_c*(2**i)
            limit = int(limit/2)
#             print('t_y_t_x',t_y,t_x)
#     im_2 = image[1,:,:]
    im_3 = np.roll(np.roll(image[2,:,:], -t_y, axis = 0), -t_x, axis = 1) 
    [r_row, r_col] = [g_coord[0]+Height+t_y, g_coord[1]+t_x]
#     print(im_1.shape, im_3.shape,  image[1,:,:].shape)
    aligned_img = np.dstack((imm3, image[1,:,:], imm1))

    return aligned_img, (b_row, b_col), (r_row, r_col)